In [1]:
import pandas as pd
import math
import numpy as np
import gc
import time

In [2]:
gc.collect()

0

In [3]:
train_mfti_df = pd.read_parquet('data/train_mfti.parquet', engine='pyarrow')

In [4]:
train_mfti_df

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,1659323026,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,1659377255,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,1659376695,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,1659376722,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,1659374929,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
...,...,...,...,...,...,...
12292583,2022-08-11,1660246479,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response
12292584,2022-08-11,1660246106,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response
12292585,2022-08-11,1660250139,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response
12292586,2022-08-11,1660250318,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response


In [5]:
#Достаем время из UNIX в event_timestamp
train_mfti_df['event_timestamp'] = pd.to_datetime(train_mfti_df['event_timestamp'], unit='s').dt.time

In [6]:
train_mfti_df

,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type
0,2022-08-01,03:03:46,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy
1,2022-08-01,18:07:35,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
2,2022-08-01,17:58:15,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
3,2022-08-01,17:58:42,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
4,2022-08-01,17:28:49,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy
...,...,...,...,...,...,...
12292583,2022-08-11,19:34:39,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response
12292584,2022-08-11,19:28:26,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response
12292585,2022-08-11,20:35:39,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response
12292586,2022-08-11,20:38:38,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response


In [7]:
#Делаем метку по тергетному действию
target_actions = ['click_response', 'preview_click_response', 'click_contacts', 'preview_click_contacts', \
                  'click_phone', 'preview_click_phone']
train_mfti_df['is_target_action'] = train_mfti_df['event_type'].apply(lambda x: 1 if x in target_actions else 0)

In [8]:
#Назначение уникального индекса для каждой пары(куки+юзер) и их дубликатов
train_mfti_df.insert(loc=0, column='common_id', \
                        value=train_mfti_df.set_index(['cookie_id','user_id']).index.factorize()[0]+1)

In [9]:
train_mfti_df

,common_id,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,is_target_action
0,1,2022-08-01,03:03:46,129850,97990f1a021d4be19aa3f955b7eacab4,951f53de61764ea0b51317200a0dbbfc,show_vacancy,0
1,2,2022-08-01,18:07:35,108347,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0
2,2,2022-08-01,17:58:15,109069,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0
3,2,2022-08-01,17:58:42,171425,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0
4,2,2022-08-01,17:28:49,252384,03bf8c511fa949c79845a5d81b09aa1d,f5a2326a17484330aa8cb4019f1b1960,show_vacancy,0
...,...,...,...,...,...,...,...,...
12292583,96470,2022-08-11,19:34:39,162851,0d3c58c6864546c689fa0997de484b30,18124998fd644e80b667a1d84bf63846,preview_click_response,1
12292584,99614,2022-08-11,19:28:26,136818,ef559274a81e4ff4b92aa3e9f6805886,5c71c1f1b7cf492ba05871c400a05250,preview_click_response,1
12292585,96452,2022-08-11,20:35:39,143721,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,1
12292586,96452,2022-08-11,20:38:38,154423,c4453128ae584dca8f2a8d20a7c74263,33c968125459420e89ffbc8cffece317,preview_click_response,1


In [10]:
#Функция для определения common_id от cookie_id для обучения рекомендательной системы
def common_from_cookie(cookie_id): # На входе получаем cookie_id
    # Вычисляем, какие common_id относятся к данному cookie_id и их частоту
    common_freq = train_mfti_df[train_mfti_df['cookie_id'] == cookie_id].common_id.value_counts(normalize=True)
    # Генерируем словарь, ключами которого будут common_id, а значениями - количество вакансий, которые 
    # мы будем использовать для рекомендательной системы (пока что расчет на то, что расчитываем 5 вакансий)
    common_ids = {com_id: round(freq/20*100) for com_id, freq in zip(common_freq.index.to_list(), common_freq.to_list())}
    return common_ids

In [11]:
common_from_cookie('00a6c5a64a274c55a836402bdeb3b2c4')

{324454: 3, 327793: 2}

In [12]:
test_public_mfti_df = pd.read_parquet('data/test_public_mfti.parquet', engine='pyarrow')

In [13]:
test_public_mfti_df

,cookie_id,vacancy_id_
0,000cd76cd33f43d4a1ac1d16d10f8bf7,"[222177, 222173, 222163, 238874, 238878, 22812..."
1,0034bc7f404341ba8412665453e7825a,"[102794, 137587, 257319, 237756, 240744, 11348..."
2,00a6c5a64a274c55a836402bdeb3b2c4,"[254292, 164602, 116438, 228634, 218819, 24065..."
3,015937a125b14e74bdff1cddc49f9172,"[246685, 138123, 115420, 210628, 212325, 235196]"
4,01de50c280794cec8804f16f45f847b7,"[219070, 251469, 166899, 212703, 214561]"
...,...,...
767,fdbcda17f22f406486837059e76c7fed,"[207851, 254989, 213344, 214180, 222146]"
768,fe6193ab26494ace9be5aae36e507618,"[115352, 230546, 225527, 120188, 109360, 23212..."
769,fe95b2826ee1452b81201ed3f4c3294d,"[240362, 114852, 253946, 251081, 127546, 244688]"
770,ff1aef256a49481698bb2e938510ff36,"[231194, 236363, 220747, 244688, 100094, 24052..."


In [14]:
train = train_mfti_df[['common_id', 'vacancy_id_', 'is_target_action']]
train

,common_id,vacancy_id_,is_target_action
0,1,129850,0
1,2,108347,0
2,2,109069,0
3,2,171425,0
4,2,252384,0
...,...,...,...
12292583,96470,162851,1
12292584,99614,136818,1
12292585,96452,143721,1
12292586,96452,154423,1


In [15]:
# Чистим датасет следующим образом: 1) убираем дубликаты, 2) убираем повторяющиеся действия каждого юзера к вакансиям,
# оставляем только 1 при наличии целевого действия и 0 при его отсутствии
non_duplicated_train = train.drop_duplicates(keep='first', ignore_index=True).sort_values(by=['common_id', 'vacancy_id_', 'is_target_action'], ascending=[True, True, False], ignore_index=True)
non_duplicated_train = non_duplicated_train.drop_duplicates(subset=['common_id', 'vacancy_id_'], keep='first', ignore_index=True)
non_duplicated_train

,common_id,vacancy_id_,is_target_action
0,1,100315,0
1,1,104757,0
2,1,105596,0
3,1,106933,1
4,1,115751,0
...,...,...,...
4723184,367637,159577,1
4723185,367638,164938,1
4723186,367639,166931,1
4723187,367640,167043,1


In [16]:
# Приводим common_id и vacancy_id к категориальному типу для дальнейшего использования матрице взаимодействий
non_duplicated_train.is_target_action = non_duplicated_train.is_target_action.astype('uint8')
non_duplicated_train.common_id = non_duplicated_train.common_id.astype('category')
non_duplicated_train.vacancy_id_ = non_duplicated_train.vacancy_id_.astype('category')
non_duplicated_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4723189 entries, 0 to 4723188
Data columns (total 3 columns):
 #   Column            Dtype   
---  ------            -----   
 0   common_id         category
 1   vacancy_id_       category
 2   is_target_action  uint8   
dtypes: category(2), uint8(1)
memory usage: 56.7 MB


# IMPLICIT

In [17]:
from scipy.sparse import csr_matrix
# Строим матрицы взаимодействий
# Vacancy_id будет представлен кодами категорий, начинающимися с 0 (то есть 100001 номер вакансии будет соответствовать
# 0 индексу кода категории, 100002 - 1 индексу и т.д.)
item_user = csr_matrix((non_duplicated_train['is_target_action'], 
                           (non_duplicated_train['vacancy_id_'].cat.codes, 
                            non_duplicated_train['common_id'].cat.codes)))

user_item = csr_matrix((non_duplicated_train['is_target_action'], 
                           (non_duplicated_train['common_id'].cat.codes, 
                            non_duplicated_train['vacancy_id_'].cat.codes)))

In [18]:
# Функция для определения метрики prec@5
def precision5(cookies, recomendations):
    ratings = []
    for cookie, y_pred in zip(cookies, recomendations):
        y_test = set(test_public_mfti_df[test_public_mfti_df['cookie_id'] == cookie]['vacancy_id_'].iloc[0])
        intersected = len(set(y_test).intersection(set(y_pred)))
        rating = intersected / len(y_pred)
        ratings.append(rating)
    return sum(ratings)/len(ratings)

# AlternatingLeastSquares

In [21]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=100, regularization=0.05, alpha=3.0, calculate_training_loss=True, random_state=42)
model.fit(user_item)

C:\Users\vitek\anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
import time
start = time.time()
cookies = test_public_mfti_df.cookie_id.to_list()
recomendations = []
for cookie in cookies:
    common_ids = common_from_cookie(cookie)
    recs = []
    for com_id, n in common_ids.items():
        if n == 0:
            continue
        filter_items = np.array(list(set(train_mfti_df[train_mfti_df['common_id'] == com_id].vacancy_id_ - 100001))) 
        # сверху вычитаем 100001 из-за разницы индексов
        ids, scores = model.recommend(com_id-1, user_item[com_id-1], N=n, filter_items=filter_items, filter_already_liked_items = True)
        recs.extend([ident+100001 for ident in ids])
    recomendations.append(recs)
end = time.time()
print(f'Потреченное время: {end-start}')

Потреченное время: 686.2077581882477


In [23]:
precision5(cookies, recomendations)

0.030051813471502535

# BayesianPersonalizedRanking

In [24]:
from implicit.bpr import BayesianPersonalizedRanking

model = BayesianPersonalizedRanking(factors=100, regularization=0.05, learning_rate=0.02, verify_negative_samples=True, random_state=42)
model.fit(user_item)

  0%|          | 0/100 [00:00<?, ?it/s]

In [25]:
import time
start = time.time()
cookies = test_public_mfti_df.cookie_id.to_list()
recomendations = []
for cookie in cookies:
    common_ids = common_from_cookie(cookie)
    recs = []
    for com_id, n in common_ids.items():
        if n == 0:
            continue
        filter_items = np.array(list(set(train_mfti_df[train_mfti_df['common_id'] == com_id].vacancy_id_ - 100001)))
        ids, scores = model.recommend(com_id-1, user_item[com_id-1], N=n, filter_items=filter_items, filter_already_liked_items = True)
        recs.extend([ident+100001 for ident in ids])
    recomendations.append(recs)
end = time.time()
print(f'Потреченное время: {end-start}')

Потреченное время: 636.1121203899384


In [26]:
precision5(cookies, recomendations)

0.01139896373056995

# LogisticMatrixFactorization

In [27]:
from implicit.lmf import LogisticMatrixFactorization

model = LogisticMatrixFactorization(factors=50, learning_rate=2, random_state=42)
model.fit(user_item)

  0%|          | 0/30 [00:00<?, ?it/s]

In [28]:
import time
start = time.time()
cookies = test_public_mfti_df.cookie_id.to_list()
recomendations = []
for cookie in cookies:
    common_ids = common_from_cookie(cookie)
    recs = []
    for com_id, n in common_ids.items():
        if n == 0:
            continue
        filter_items = np.array(list(set(train_mfti_df[train_mfti_df['common_id'] == com_id].vacancy_id_ - 100001)))
        ids, scores = model.recommend(com_id-1, user_item[com_id-1], N=n, filter_items=filter_items, filter_already_liked_items = True)
        recs.extend([ident+100001 for ident in ids])
    recomendations.append(recs)
end = time.time()
print(f'Потреченное время: {end-start}')

Потреченное время: 625.3219749927521


In [29]:
precision5(cookies, recomendations)

0.017357512953367862

In [19]:
from implicit.als import AlternatingLeastSquares

model = AlternatingLeastSquares(factors=100, regularization=0.05, alpha=3.0, calculate_training_loss=True, random_state=42)
model.fit(user_item)

C:\Users\vitek\anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [20]:
# Создание списка неактульных вакансий, с которомы давно не было взаимодействий
# 78 973 вакансии не были провзаимодействованы за последние 2 недели рассматриваемого периода (prec = 0.031)
# 99 046 вакансии не были провзаимодействованы за последнюю неделю (prec = 0.033)
# 117 944 вакансии не были провзаимодействованы за последние 3 дня (prec = 0.036)
# 127 405 вакансии не были провзаимодействованы за последние 2 дня (prec = 0.038)
# 140 242 вакансии не были провзаимодействованы за последний 1 день - 0.04
non_valid_vacs = []
for vac, group in train_mfti_df.groupby('vacancy_id_'):
    max_date = group['event_date'].max()
    if max_date <= '2022-09-30':
        non_valid_vacs.append(vac)
len(non_valid_vacs)

140242

In [21]:
import time
start = time.time()
cookies = test_public_mfti_df.cookie_id.to_list()
non_valid_vacs_recommend = list(map(lambda x: x - 100001, non_valid_vacs))
recomendations = []
for cookie in cookies:
    common_ids = common_from_cookie(cookie)
    recs = []
    for com_id, n in common_ids.items():
        if n == 0:
            continue
        actioned_vacs = train_mfti_df[train_mfti_df['common_id'] == com_id].vacancy_id_.unique()
        actioned_vacs = actioned_vacs - 100001
        filter_items = np.concatenate((actioned_vacs, non_valid_vacs_recommend))
        ids, scores = model.recommend(com_id-1, user_item[com_id-1], N=n, filter_items=filter_items, filter_already_liked_items = True)
        recs.extend([ident+100001 for ident in ids])
    recomendations.append(recs)
end = time.time()
print(f'Потреченное время: {end-start}')

Потреченное время: 601.2093601226807


In [22]:
precision5(cookies, recomendations)

0.040414507772020644

# Ниже пока не актуально
# -----------------------------------------------------------------------------------------------------------

In [119]:
#Создаем таблицу с рейтингом вакансий, рассчитанным как доля целевых действий от общего количества взаимодействий
vacancies = train_mfti_df.groupby(by='vacancy_id_', sort=False).agg(counts=('common_id', 'count'), \
                                                        target_actions=('is_target_action', 'sum'))
vacancies['rating'] = round(vacancies.target_actions / vacancies.counts, 2)
vacancies

,counts,target_actions,rating
vacancy_id_,,,
129850,49,1,0.02
108347,1234,78,0.06
109069,1763,199,0.11
171425,1541,152,0.10
252384,2877,225,0.08
...,...,...,...
135208,1,1,1.00
165045,1,1,1.00
144452,1,1,1.00


In [120]:
#Топ вакансий от 200 целевых действий
top_vacancies = vacancies[vacancies.target_actions > 200].sort_values(by='rating', ascending=False)
top_vacancies

,counts,target_actions,rating
vacancy_id_,,,
184067,468,212,0.45
106282,1110,268,0.24
115589,3059,694,0.23
203115,1357,305,0.22
205630,2400,528,0.22
...,...,...,...
168935,7815,340,0.04
111592,7559,249,0.03
258441,24253,624,0.03


In [121]:
#Соединяем таблицы трейна и теста по совпадающим cookie_id
merged_df = train_mfti_df.merge(test_public_mfti_df, how='inner', on='cookie_id', suffixes=('train', 'pred'))

In [122]:
#Выводим данные из merged_df с условием на наличие целевого действия
merged_df1 = merged_df[merged_df.is_target_action == 1]
merged_df1

,common_id,event_date,event_timestamp,vacancy_id_train,cookie_id,user_id,event_type,is_target_action,vacancy_id_pred
347,470,2022-08-08,10:49:08,110667,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,click_contacts,1,"[156414, 240132, 230487, 242334, 238557, 11355..."
348,470,2022-08-08,08:26:48,141369,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,click_contacts,1,"[156414, 240132, 230487, 242334, 238557, 11355..."
349,470,2022-08-08,10:26:09,156048,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,click_contacts,1,"[156414, 240132, 230487, 242334, 238557, 11355..."
350,470,2022-08-02,11:52:42,110667,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,click_response,1,"[156414, 240132, 230487, 242334, 238557, 11355..."
351,470,2022-08-02,11:16:42,138123,434ddd68eaea4cceafb83708dec13b18,e4fcc60c6f50486bb7461f09b029fd90,click_response,1,"[156414, 240132, 230487, 242334, 238557, 11355..."
...,...,...,...,...,...,...,...,...,...
79017,362535,2022-09-30,18:19:16,207423,8558e065d8f847558060e258f7c9e17f,c0856dd3f509497bbfdb68ae93761f54,click_response,1,"[116438, 212027, 228866, 236742, 102919, 11150..."
79018,362535,2022-09-30,17:54:07,222030,8558e065d8f847558060e258f7c9e17f,c0856dd3f509497bbfdb68ae93761f54,click_response,1,"[116438, 212027, 228866, 236742, 102919, 11150..."
79019,362535,2022-09-30,17:59:54,234769,8558e065d8f847558060e258f7c9e17f,c0856dd3f509497bbfdb68ae93761f54,click_response,1,"[116438, 212027, 228866, 236742, 102919, 11150..."
79020,362535,2022-09-30,18:07:09,238065,8558e065d8f847558060e258f7c9e17f,c0856dd3f509497bbfdb68ae93761f54,click_response,1,"[116438, 212027, 228866, 236742, 102919, 11150..."


In [123]:
merged_df1.groupby(by=['common_id', 'vacancy_id_train'])['is_target_action'].agg('sum')

common_id  vacancy_id_train
470        110667              2
           138123              1
           141369              2
           156048              2
           233922              2
                              ..
365819     216558              1
366922     213923              2
367028     157470              2
367266     138622              1
367571     256132              1
Name: is_target_action, Length: 8096, dtype: int64

In [124]:
merged_df1[merged_df1['common_id'] == 367571]

,common_id,event_date,event_timestamp,vacancy_id_train,cookie_id,user_id,event_type,is_target_action,vacancy_id_pred
36582,367571,2022-08-20,14:50:20,256132,67718455d0814f3bab17774ab7809aff,None,preview_click_response,1,"[247309, 219266, 198542, 244688, 222836, 22765..."


In [125]:
train_mfti_df[(train_mfti_df['vacancy_id_'].isin([256132, 247309])) & (train_mfti_df['is_target_action'] == 1)]

,common_id,event_date,event_timestamp,vacancy_id_,cookie_id,user_id,event_type,is_target_action
10983878,121671,2022-08-17,10:26:15,256132,80fe6fed33364daebd70e6e2759a8aa9,a6b029cc50874160a8fc7a32d70661c5,click_contacts,1
10983879,3298,2022-08-17,11:56:25,256132,c43565885ec8432a99e3caf86839612a,None,click_contacts,1
10986798,130023,2022-08-17,15:56:51,256132,e186c538a9b844b9853f6cd45a4b90f1,68db61a8467b4c8581c6fb4f7dcdb634,click_contacts,1
10994960,130023,2022-08-17,15:56:53,256132,e186c538a9b844b9853f6cd45a4b90f1,68db61a8467b4c8581c6fb4f7dcdb634,click_phone,1
10998401,314026,2022-09-19,17:37:35,247309,2ce5faa0869e46f683e7c3784c80c9df,None,click_contacts,1
...,...,...,...,...,...,...,...,...
12291606,98188,2022-08-11,17:28:05,256132,9955e537690a4fb0a3ca39fd55786450,7b2683749ab441858b619a2b3ffc6920,preview_click_response,1
12291981,67133,2022-08-11,07:18:00,256132,a1359fdb697d45849cb362d5c86b7f81,2db3184a6d614106896174c74d9e5b3f,preview_click_response,1
12292262,76227,2022-08-11,12:31:44,256132,9f565543a21440709766c755f498ded5,4bafa8daba8a4d0b8ef76fb0fad8550c,preview_click_response,1
12292307,98324,2022-08-11,22:06:03,256132,89a093ed27e0441388fc72e5f9125e79,6ea900b3aca543368ed52690cff7d7b6,preview_click_response,1


In [9]:
test_private_users_mfti_df = pd.read_parquet('data/test_private_users_mfti.parquet', engine='pyarrow')

In [10]:
test_private_users_mfti_df

,cookie_id
0,0018914ba3e54011b28fa715583d3354
1,0035c298d8c64f368ae730a9cca9bb20
2,00956458877448ec9fba87fb97443fdf
3,0099387c921b41e7bae6c99dd8254b60
4,009f65e8ae99413a8da94a491320580a
...,...
3081,ffadd195859444d2ade2479b0611c5c1
3082,ffbc08b528c64f22996873fc63872202
3083,ffdeaf3c34544529880aebf17c103f6c
3084,ffefa79a74804ee69e6c131e0d05b948


In [11]:
test_private_sample_submission_mfti_df = pd.read_parquet('data/test_private_sample_submission_mfti.parquet', engine='pyarrow')

In [12]:
test_private_sample_submission_mfti_df

,cookie_id,predictions
0,0018914ba3e54011b28fa715583d3354,"[100100, 100101, 100102, 100103, 100104]"
1,0035c298d8c64f368ae730a9cca9bb20,"[100100, 100101, 100102, 100103, 100104]"
2,00956458877448ec9fba87fb97443fdf,"[100100, 100101, 100102, 100103, 100104]"
3,0099387c921b41e7bae6c99dd8254b60,"[100100, 100101, 100102, 100103, 100104]"
4,009f65e8ae99413a8da94a491320580a,"[100100, 100101, 100102, 100103, 100104]"
...,...,...
3081,ffadd195859444d2ade2479b0611c5c1,"[100100, 100101, 100102, 100103, 100104]"
3082,ffbc08b528c64f22996873fc63872202,"[100100, 100101, 100102, 100103, 100104]"
3083,ffdeaf3c34544529880aebf17c103f6c,"[100100, 100101, 100102, 100103, 100104]"
3084,ffefa79a74804ee69e6c131e0d05b948,"[100100, 100101, 100102, 100103, 100104]"


In [13]:
train_mfti_df.vacancy_id_.value_counts()

260154    59105
198114    50187
203404    45634
202608    41620
116823    40023
          ...  
208174        1
207219        1
208297        1
207097        1
136818        1
Name: vacancy_id_, Length: 160167, dtype: int64

In [14]:
train_mfti_df.cookie_id.value_counts()

924398d361a0454c8c30845c2b4c5747    4898
7d8cc5a0fef44378a2d90a237cda288e    3343
5934d5b8a0b348829d8efabe69c733eb    3315
b7dd2f20fdd6472ab62f8d86a739cd5d    3312
353b56c4fa6447d0ba0f08b42d86e51f    2795
                                    ... 
1b4240c448df47f29e00ef38b8215ea2       5
99c776f7ff1d4e73919342365bc0de0a       5
f84c882a92d74646bfc6eb4447b11d3c       5
9b86c05707bc43d19dfaf60563627c2d       5
694ea921f0d14d56853bf9856866110d       5
Name: cookie_id, Length: 330180, dtype: int64

In [ ]:
def general_func(cookie_id, n_vacancies=5):
    